In [1]:
from openai import OpenAI

In [20]:
client = OpenAI()

Upload training file to OpenAI server

In [54]:
client.files.create(file=open("dataset/train.jsonl", 'rb'), purpose="fine-tune")

FileObject(id='file-FJ4XYdangD22sfRoxPwHsP', bytes=168281, created_at=1762325996, filename='train.jsonl', object='file', purpose='fine-tune', status='processed', expires_at=None, status_details=None)

In [55]:
client.files.list()

SyncCursorPage[FileObject](data=[FileObject(id='file-FJ4XYdangD22sfRoxPwHsP', bytes=168281, created_at=1762325996, filename='train.jsonl', object='file', purpose='fine-tune', status='processed', expires_at=None, status_details=None)], has_more=False, object='list', first_id='file-FJ4XYdangD22sfRoxPwHsP', last_id='file-FJ4XYdangD22sfRoxPwHsP')

Retrieving a file

In [70]:
client.files.retrieve("file-FJ4XYdangD22sfRoxPwHsP") 

FileObject(id='file-FJ4XYdangD22sfRoxPwHsP', bytes=168281, created_at=1762325996, filename='train.jsonl', object='file', purpose='fine-tune', status='processed', expires_at=None, status_details=None)

Creating a fine-tuning Job

Parameters:
model -> Pre-trained GPT model name which you want to fine-tune.

training-file -> ID of an uploaded file that contains training data in JSONL or Toon format.

hyperparameters -> An object containing the hyperparameters to be used in the fine-tuning job.

    - batch_size -> Set the number of examples in each batch. Defaults to auto.

    - learning_rate_multiplier -> Scaling factor for the learning rate. Defaults to auto.

    - n_epochs -> Refers to the number of epochs to train the model for. Defaults to auto.

Suffix -> A string of upto 18 characters to be added to fine-tuned name.

Seed -> An integer value used to control the reproducibility of the job.

Validation_file -> ID of an uploaded file that contains validation data in JSONL or Toon format.



In [71]:
client.fine_tuning.jobs.create(training_file="file-FJ4XYdangD22sfRoxPwHsP",
                               model="gpt-3.5-turbo",
                               hyperparameters={
                                   "n_epochs": 4
                               })

FineTuningJob(id='ftjob-fWvSY4vmL0MrW3Q6KlJS6uwr', created_at=1762328402, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs=4), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-hbYKtyDbqYrnHlOyq41rI1AP', result_files=[], seed=724099736, status='validating_files', trained_tokens=None, training_file='file-FJ4XYdangD22sfRoxPwHsP', validation_file=None, estimated_finish=None, integrations=[], metadata=None, method=Method(type='supervised', dpo=None, reinforcement=None, supervised=SupervisedMethod(hyperparameters=SupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs=4))), user_provided_suffix=None, usage_metrics=None, shared_with_openai=False, eval_id=None)

In [21]:
client.fine_tuning.jobs.list(limit=3)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-fWvSY4vmL0MrW3Q6KlJS6uwr', created_at=1762328402, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal::CYSqhFvt', finished_at=1762330081, hyperparameters=Hyperparameters(batch_size=1, learning_rate_multiplier=2.0, n_epochs=4), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-hbYKtyDbqYrnHlOyq41rI1AP', result_files=['file-VXe9hjagwHC84UWdYgQxpM'], seed=724099736, status='succeeded', trained_tokens=126020, training_file='file-FJ4XYdangD22sfRoxPwHsP', validation_file=None, estimated_finish=None, integrations=[], metadata=None, method=Method(type='supervised', dpo=None, reinforcement=None, supervised=SupervisedMethod(hyperparameters=SupervisedHyperparameters(batch_size=1, learning_rate_multiplier=2.0, n_epochs=4))), user_provided_suffix=None, usage_metrics=None, shared_with_openai=False, eval_id=None), FineTuningJob(id='ftjob-uFX34aYqyOcyrAEyioo1Yegt', create

To retrieve the information about the job

In [3]:
retrieved_job = client.fine_tuning.jobs.retrieve("ftjob-fWvSY4vmL0MrW3Q6KlJS6uwr")
print(retrieved_job)
if retrieved_job.status == "succeeded":
    fine_tuned_model_name = retrieved_job.fine_tuned_model
    print(f"Fine-tuned model name: {fine_tuned_model_name}")
else:
    print(f"Fine-tuning job status: {retrieved_job.status}. Model name not yet available.")

FineTuningJob(id='ftjob-fWvSY4vmL0MrW3Q6KlJS6uwr', created_at=1762328402, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal::CYSqhFvt', finished_at=1762330081, hyperparameters=Hyperparameters(batch_size=1, learning_rate_multiplier=2.0, n_epochs=4), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-hbYKtyDbqYrnHlOyq41rI1AP', result_files=['file-VXe9hjagwHC84UWdYgQxpM'], seed=724099736, status='succeeded', trained_tokens=126020, training_file='file-FJ4XYdangD22sfRoxPwHsP', validation_file=None, estimated_finish=None, integrations=[], metadata=None, method=Method(type='supervised', dpo=None, reinforcement=None, supervised=SupervisedMethod(hyperparameters=SupervisedHyperparameters(batch_size=1, learning_rate_multiplier=2.0, n_epochs=4))), user_provided_suffix=None, usage_metrics=None, shared_with_openai=False, eval_id=None)
Fine-tuned model name: ft:gpt-3.5-turbo-0125:personal::CYSqhFvt


To get the events that are running inside a job

In [5]:
client.fine_tuning.jobs.list_events(fine_tuning_job_id="ftjob-fWvSY4vmL0MrW3Q6KlJS6uwr", limit=5)


SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-pB1EiVVWUDOKyVWUaf2zmVBS', created_at=1762362930, level='info', message='The job has successfully completed', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-UUcIS4hJHjLYHXnRM8m8uACs', created_at=1762362924, level='info', message='Usage policy evaluations completed, model is now enabled for sampling', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-0HfOsTyJVDMv7jDRDB3QTOaL', created_at=1762362924, level='info', message='Moderation checks for snapshot ft:gpt-3.5-turbo-0125:personal::CYSqhFvt passed.', object='fine_tuning.job.event', data={'blocked': False, 'results': [{'flagged': False, 'category': 'harassment/threatening', 'enforcement': 'blocking'}, {'flagged': False, 'category': 'sexual', 'enforcement': 'blocking'}, {'flagged': False, 'category': 'sexual/minors', 'enforcement': 'blocking'}, {'flagged': False, 'category': 'propaganda

Testing the fine-tuned model

In [24]:
from openai import OpenAI
client = OpenAI()




In [13]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
     {"role": "system", "content": "You are an expert English Translator and Improver."},
    {"role": "user", "content": "ich liebe das programmieren und das lernen von neuen dingen"}
  ]
)

print(completion.choices[0].message.content)

I love programming and learning new things.


In [15]:
completion = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0125:personal::CYSqhFvt",
  messages=[
     {"role": "system", "content": "English Translator and Improver"},
    {"role": "user", "content": "ich liebe das programmieren und das lernen von neuen dingen"}
  ]
)

print(completion.choices[0].message.content)

As an English Translator and Improver, provide a clear and helpful answer.


Delete your fine-tuned model

In [23]:
model_id = "ft:gpt-3.5-turbo-0125:personal::CYSqhFvt"

response = client.models.delete(model_id)
print(response)

ModelDeleted(id='ft:gpt-3.5-turbo-0125:personal::CYSqhFvt', deleted=True, object='model')
